In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np

In [2]:
X_train_prep = np.loadtxt("imdb_X_train_prep.csv", delimiter=",")
X_test_prep = np.loadtxt("imdb_X_test_prep.csv", delimiter=",")
y_train_prep = np.loadtxt("imdb_y_train_prep.csv", delimiter=",")
y_test_prep = np.loadtxt("imdb_y_test_prep.csv", delimiter=",")

# Load the data

In [9]:
X_train_prep.shape

(25000, 200)

In [3]:
results_dict = {}

# Parameters tuning experiments

In [22]:
# Initiate the model

num_hidden_layers = 1
nodes_hidden_layer = 5
embedding_dimension = 16
# num_filters_conv = [2**2, 2**5, 2**7, 2**8]
# kernel_conv = [1, 3, 5, 7, 9]
optimizers = ['RMSprop', 'sgd']
# embedding_dimension = 16
num_filters_conv = 32
kernel_conv = 3
# optimizer = 'adam'

for optimizer in optimizers:
    
    model = tf.keras.Sequential()
    model.add(layers.Embedding(input_dim=25000, output_dim=embedding_dimension,input_length=X_train_prep.shape[1]))
    model.add(layers.Conv1D(filters=num_filters_conv, kernel_size=kernel_conv, activation='relu'))
    model.add(layers.MaxPooling1D())
    model.add(layers.LSTM(nodes_hidden_layer))
    model.add(layers.Dense(nodes_hidden_layer))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizer, loss='binary_crossentropy')
    model.summary()
    # Fit the model & Predict
    model.fit(X_train_prep, y_train_prep)
    y_pred = model.predict(X_test_prep)

    from sklearn.metrics import accuracy_score 
    accuracy = accuracy_score(y_test_prep, np.round(y_pred))
    print(f"Accuracy Score on the test set: {accuracy}")
    results_dict[f'{num_hidden_layers}_hiddenLayer_with_{nodes_hidden_layer}_nodes-{embedding_dimension}_output_dim-{num_filters_conv}filters-{kernel_conv}kernels-{optimizer}'] = accuracy

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, 200, 16)           400000    
                                                                 
 conv1d_19 (Conv1D)          (None, 198, 32)           1568      
                                                                 
 max_pooling1d_19 (MaxPoolin  (None, 99, 32)           0         
 g1D)                                                            
                                                                 
 lstm_18 (LSTM)              (None, 5)                 760       
                                                                 
 dense_36 (Dense)            (None, 5)                 30        
                                                                 
 dense_37 (Dense)            (None, 1)                 6         
                                                     

In [23]:
results_dict

{'1_hiddenLayer_with_5_nodes-16_output_dim-1filters-1kernels-adam': 0.80032,
 '2_hiddenLayer_with_5_nodes-16_output_dim-1filters-1kernels-adam': 0.785,
 '2_hiddenLayer_with_16_nodes-16_output_dim-1filters-1kernels-adam': 0.77244,
 '2_hiddenLayer_with_32_nodes-16_output_dim-1filters-1kernels-adam': 0.77904,
 '1_hiddenLayer_with_16_nodes-16_output_dim-1filters-1kernels-adam': 0.78016,
 '1_hiddenLayer_with_32_nodes-16_output_dim-1filters-1kernels-adam': 0.78556,
 '1_hiddenLayer_with_5_nodes-32_output_dim-1filters-1kernels-adam': 0.76288,
 '1_hiddenLayer_with_5_nodes-64_output_dim-1filters-1kernels-adam': 0.74248,
 '1_hiddenLayer_with_5_nodes-16_output_dim-4filters-1kernels-adam': 0.85924,
 '1_hiddenLayer_with_5_nodes-16_output_dim-32filters-1kernels-adam': 0.8466,
 '1_hiddenLayer_with_5_nodes-16_output_dim-128filters-1kernels-adam': 0.82752,
 '1_hiddenLayer_with_5_nodes-16_output_dim-256filters-1kernels-adam': 0.85992,
 '1_hiddenLayer_with_5_nodes-16_output_dim-32filters-3kernels-adam': 0

# add Bidirectional wrapper layer

In [24]:
# Before adding the bidirectional layer
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=25000, output_dim=32, input_length=X_train_prep.shape[1]))
model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=7, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D())
model.add(tf.keras.layers.LSTM(64))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

model.fit(X_train_prep, y_train_prep)

y_pred = model.predict(X_test_prep)

print(f"Accuracy Score on the test set: {accuracy_score(y_test_prep, np.round(y_pred))}")

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_21 (Embedding)    (None, 200, 32)           800000    
                                                                 
 conv1d_21 (Conv1D)          (None, 194, 128)          28800     
                                                                 
 max_pooling1d_21 (MaxPoolin  (None, 97, 128)          0         
 g1D)                                                            
                                                                 
 lstm_20 (LSTM)              (None, 64)                49408     
                                                                 
 dense_40 (Dense)            (None, 10)                650       
                                                                 
 dense_41 (Dense)            (None, 10)                110       
                                                     

In [25]:
# After adding the bidirectional layer
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=25000, output_dim=32, input_length=X_train_prep.shape[1]))
model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=7, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D())
model.add(layers.Bidirectional(layers.LSTM(64)))    # Bidirectional layer
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

model.fit(X_train_prep, y_train_prep)

y_pred = model.predict(X_test_prep)

print(f"Accuracy Score on the test set: {accuracy_score(y_test_prep, np.round(y_pred))}")

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 200, 32)           800000    
                                                                 
 conv1d_22 (Conv1D)          (None, 194, 128)          28800     
                                                                 
 max_pooling1d_22 (MaxPoolin  (None, 97, 128)          0         
 g1D)                                                            
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_43 (Dense)            (None, 10)                1290      
                                                                 
 dense_44 (Dense)            (None, 10)              

# Removing Conv layer - experiment

In [27]:
# Removing Convolutional layer
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=25000, output_dim=32, input_length=X_train_prep.shape[1]))
# model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=7, activation='relu'))
# model.add(tf.keras.layers.MaxPooling1D())
# model.add(layers.Bidirectional(layers.LSTM(64)))    # Bidirectional layer
model.add(layers.LSTM(64))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

model.fit(X_train_prep, y_train_prep)

y_pred = model.predict(X_test_prep)

print(f"Accuracy Score on the test set: {accuracy_score(y_test_prep, np.round(y_pred))}")

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, 200, 32)           800000    
                                                                 
 lstm_22 (LSTM)              (None, 64)                24832     
                                                                 
 dense_49 (Dense)            (None, 10)                650       
                                                                 
 dense_50 (Dense)            (None, 10)                110       
                                                                 
 dense_51 (Dense)            (None, 1)                 11        
                                                                 
Total params: 825,603
Trainable params: 825,603
Non-trainable params: 0
_________________________________________________________________
782/782 [==============================] - 18s 